In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("..")

In [2]:
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Loading Parameters

In [ ]:
llm_model = "gpt-3.5-turbo"